In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
import sklearn.model_selection as model_selection
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# from sklearn.utils import shuffle
# import stop_words
TRAIN_LABELS = 'participants/train/labels/labels.csv'
TRAIN_TEXT = 'participants/train/extracted_data/extract_combined.csv'
TEST_TEXT = 'participants/test/extracted_data/extract_combined.csv'
TRAIN_ORG_TEXT = 'participants/get_only_ORG.csv'

In [6]:
train_labels_df = pd.read_csv(TRAIN_LABELS, usecols=['document_name','is_fitara'])
train_text_df = pd.read_csv(TRAIN_TEXT)
test_df = pd.read_csv(TEST_TEXT)


In [7]:
train_labels_df.is_fitara = train_labels_df.is_fitara.eq('Yes').mul(1)

train_df = pd.merge(
    train_labels_df, 
    train_text_df, 
    on='document_name', 
    how='inner'
)
train_df.head()

,document_name,is_fitara,text
0,04-42RFP.pdf,0,\n \nOMB No. 0990-0115 \n \n Electronic Requ...
1,1_-_Brand_name_JOFOC_1782798.pdf,0,\n \nJustification for Other than Full and Op...
2,1_-_Brand_Name_JOFOC_for_FBO.docx,0,Justification for Other than Full and Open Com...
3,1_-_Brand_Name_Only.docx,1,Justification for Other than Full and Open Com...
4,1_-_JOFOC_-_FSSI_Limited_Source.doc,1,Attachment C\r\r\rHHS Template and Instruction...


In [156]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        return: modified initial string
    """
    text = text.lower()  # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)  # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text)  # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
    text = text.replace('x', '')
    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)  # remove stopwors from text
    return text

In [152]:
# nltk.download('stopwords')

In [153]:
train_df['text'] = train_df['text'].apply(clean_text)

In [7]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(train_df.text, train_df.is_fitara, test_size=0.3)

In [220]:
tfidf_vect_ngram = TfidfVectorizer( max_df=0.7, analyzer='word', ngram_range=(2,3), token_pattern='\w{1,}' ,max_features=50000)
#stop_words='english',
#, token_pattern=r'\w{1,}'
tfidf_vect_ngram.fit(train_x)
ngram_train = tfidf_vect_ngram.transform(train_x.values)
ngram_test = tfidf_vect_ngram.transform(test_x.values)

In [228]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
# Loss 4.45
nb_classifier2 = SGDClassifier(max_iter=100, verbose=1)#class_weight={ 0:0.3, 1:0.7 })
# for i in range(3):
nb_classifier2.fit(ngram_train, train_y)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


-- Epoch 1
Norm: 79.02, NNZs: 49737, Bias: -1.064832, T: 668, Avg. loss: 0.718223
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 68.03, NNZs: 49864, Bias: -1.320871, T: 1336, Avg. loss: 0.190823
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 58.95, NNZs: 49882, Bias: -1.212988, T: 2004, Avg. loss: 0.089696
Total training time: 0.02 seconds.
-- Epoch 4
Norm: 53.24, NNZs: 49905, Bias: -1.332375, T: 2672, Avg. loss: 0.064695
Total training time: 0.03 seconds.
-- Epoch 5
Norm: 49.03, NNZs: 49911, Bias: -1.229155, T: 3340, Avg. loss: 0.051757
Total training time: 0.03 seconds.
-- Epoch 6
Norm: 44.92, NNZs: 49930, Bias: -1.184454, T: 4008, Avg. loss: 0.036661
Total training time: 0.04 seconds.
-- Epoch 7
Norm: 42.83, NNZs: 49937, Bias: -1.147682, T: 4676, Avg. loss: 0.030618
Total training time: 0.04 seconds.
-- Epoch 8
Norm: 40.28, NNZs: 49953, Bias: -1.147056, T: 5344, Avg. loss: 0.027815
Total training time: 0.05 seconds.
-- Epoch 9
Norm: 38.31, NNZs: 49959, Bias: -1.129807, T: 

Norm: 27.11, NNZs: 49996, Bias: -1.001385, T: 64796, Avg. loss: 0.005708
Total training time: 0.41 seconds.
-- Epoch 98
Norm: 27.14, NNZs: 49996, Bias: -0.999866, T: 65464, Avg. loss: 0.006010
Total training time: 0.42 seconds.
-- Epoch 99
Norm: 27.14, NNZs: 49996, Bias: -0.996900, T: 66132, Avg. loss: 0.005843
Total training time: 0.42 seconds.
-- Epoch 100
Norm: 27.05, NNZs: 49996, Bias: -0.999863, T: 66800, Avg. loss: 0.005352
Total training time: 0.43 seconds.


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=100,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=1, warm_start=False)

In [229]:
# Loss 6.98
# nb_classifier2 = MultinomialNB(class_prior=[.5, .5])
# nb_classifier2.fit(ngram_train, train_y)
# ngram_pred = nb_classifier2.predict(ngram_test)
# ngram_score = metrics.accuracy_score(test_y, ngram_pred)

In [230]:

ngram_pred = nb_classifier2.predict(ngram_test)
ngram_score = metrics.accuracy_score(test_y, ngram_pred)
print('============= NGRAM VECTORIZER ================')
print('Loss = ',metrics.log_loss(test_y, ngram_pred))
print('Score = ',ngram_score)
print('CM = \n',metrics.confusion_matrix(test_y, ngram_pred, labels=[0, 1]))

============= NGRAM VECTORIZER ================
Loss =  4.813819679956798
Score =  0.8606271777003485
CM = 
 [[190  19]
 [ 21  57]]


### Spacy

In [8]:
import spacy
nlp = spacy.load('en')


In [6]:
doc = nlp(train_df.text[3])

In [7]:
labels = set()
for ent in doc.ents:
    labels.add(ent.label_)
    if ent.label_ == 'ORG':
        print(ent.text, ent.label_)

Norovirus Therapeutics ORG
Vaccines Total ORG
ETA ORG
Noroviruses ORG
KingFisher Flex Purification System ORG
KingFisher Flex Purification System ORG
Manufacturer is Life Technologies Corporation ORG
Thermo Fisher Scientific ORG
Grand ORG
RNA ORG
USC ORG
FSS ORG
NIH ORG
Contracting Officer ORG
USC ORG
Unusual & ORG
USC ORG
USC ORG
USC ORG
USC ORG
USC ORG
Life Technologies ORG


In [27]:
doc2 = nlp(train_df.text[15])

In [5]:
for ent in doc2.ents:
    if ent.label_ == 'ORG':
        print(ent.text, ent.label_)

NameError: name 'doc2' is not defined

In [9]:
count = 0
def get_only_ORG(text):
    global count
    doc = nlp(text)
    ORGS = ""
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            ORGS += "\t"
            ORGS += ent.text
    count += 1
    print(count)
    return ORGS

# count = 0
train_df['text'] = train_df['text'].apply(get_only_ORG)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [10]:
train_df.to_csv('participants/get_only_ORG.csv', sep=',', index=False)